In [1]:
import sqlite3

In [2]:
db = sqlite3.connect("content.db")

In [3]:
print(db.execute("select sqlite_version()").fetchall())

[('3.40.0',)]


In [4]:
for row in db.execute("SELECT * FROM plugin_repos LIMIT 10"):
    print(row)

('simonw/datasette-ripgrep', '["Search"]', None)
('simonw/datasette-graphql', '["API", "Template"]', None)
('simonw/datasette-cluster-map', '["Visualization", "Geospatial", "Leaflet", "Maps"]', None)
('simonw/datasette-vega', '["Visualization"]', 'plot chart graph')
('simonw/datasette-auth-github', '["Authentication"]', None)
('simonw/datasette-json-html', '["Render Cell"]', None)
('simonw/datasette-render-images', '["Render Cell"]', None)
('simonw/datasette-css-properties', '["Output"]', None)
('simonw/datasette-copyable', '["Output"]', None)
('simonw/datasette-media', '["Ops"]', None)


In [5]:
db.row_factory = sqlite3.Row

In [6]:
for row in db.execute("SELECT * FROM plugin_repos LIMIT 10"):
    print(row)

In [7]:
for row in db.execute("SELECT * FROM plugin_repos LIMIT 10"):
    print(dict(row))

{'repo': 'simonw/datasette-ripgrep', 'tags': '["Search"]', 'extra_search': None}
{'repo': 'simonw/datasette-graphql', 'tags': '["API", "Template"]', 'extra_search': None}
{'repo': 'simonw/datasette-cluster-map', 'tags': '["Visualization", "Geospatial", "Leaflet", "Maps"]', 'extra_search': None}
{'repo': 'simonw/datasette-vega', 'tags': '["Visualization"]', 'extra_search': 'plot chart graph'}
{'repo': 'simonw/datasette-auth-github', 'tags': '["Authentication"]', 'extra_search': None}
{'repo': 'simonw/datasette-json-html', 'tags': '["Render Cell"]', 'extra_search': None}
{'repo': 'simonw/datasette-render-images', 'tags': '["Render Cell"]', 'extra_search': None}
{'repo': 'simonw/datasette-css-properties', 'tags': '["Output"]', 'extra_search': None}
{'repo': 'simonw/datasette-copyable', 'tags': '["Output"]', 'extra_search': None}
{'repo': 'simonw/datasette-media', 'tags': '["Ops"]', 'extra_search': None}


In [8]:
db.execute("""
           create table peps (
               id integer primary key,
               title text,
               author text,
               status text,
               type text,
               created text,
               body text
           );
           """)

In [9]:
def parse_pep(s):
    intro, body = s.split("\n\n", 1)
    pep = {}
    current_key = None
    current_value = None
    for line in intro.split("\n"):
        # If the line starts with whitespace, it's a continuation of the previous value
        if line.startswith(" ") or line.startswith("\t"):
            if current_key is not None:
                current_value += " " + line.strip()
                pep[current_key] = current_value.strip()
        else:
            # Split the line into key and value
            parts = line.split(": ", 1)
            if len(parts) == 2:
                key, value = parts
                # Update the current key and value
                current_key = key
                current_value = value
                # Add the key-value pair to the pep dictionary
                pep[current_key] = current_value.strip()
    pep["Body"] = body.strip()
    return pep

In [10]:
import urllib

In [12]:
zen = urllib.request.urlopen(
    "https://raw.githubusercontent.com/python/peps/main/pep-0020.txt"
).read().decode("utf-8")

pep = parse_pep(zen)

In [13]:
db.execute("""
           insert into peps (
               id, title, author, status, type, created, body
           ) values (
               ?, ?, ?, ?, ?, ?, ?
           )
           """, (
               pep["PEP"],
               pep["Title"],
               pep["Author"],
               pep["Status"],
               pep["Type"],
               pep["Created"],
               pep["Body"],
           ))

In [14]:
print(db.execute("select * from peps").fetchall())

In [15]:
with db:
    db.execute("""
               update peps set author = ?
               where id = ?
               """, ["Tim Peters", 20])

In [16]:
print(db.execute("select * from peps").fetchall())